In [69]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pyclipper
from ast import literal_eval
from tqdm import tqdm
from shapely.geometry import Polygon, LineString
import shapely.wkt
from collections import Counter, OrderedDict
from sklearn.preprocessing import MinMaxScaler

tqdm.pandas()

c:\users\rianne\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Load dataframe with information from Amsterdam oud_oost

In [18]:
info_city = pd.read_csv("oud_oost_original.csv")
info_city["pand_id"] = info_city["pand_id"].apply(str)

Wall time: 2.02 s


In [19]:
def get_info(group):
    # een adress in het pand
    if len(group) == 1:
        # staart
        straat = str(group.openbareRuimteNaam.values[0])
        
        # huisnummer
        huisnummers = str(int(group.huisnummer.values[0]))
        
        # postcode
        try:
            postcode = str(group.postcode.values[0])
        except:
            postcode = "unknown"
    
    # meerdere adressen in een pand
    else:
        # straat
        alle_straten = group.openbareRuimteNaam.unique()
        if len(alle_straten) == 1:
            straat = alle_straten[0]
        else:
            straat = str(group.openbareRuimteNaam.mode().values[0])
        
        # huisnummer
        alle_huisnummers = group.huisnummer.unique()
        if len(alle_huisnummers) == 1:
            huisnummers = str(int(alle_huisnummers[0]))
        else:
            huisnummers = str(int(min(alle_huisnummers))) + "-" + str(int(max(alle_huisnummers)))
            
        # postcode
        alle_postcodes = group.postcode.unique()
        if len(alle_postcodes) == 1:
            postcode = str(alle_postcodes[0])    
        else:
            print("meer dan een postcode")
    
    return str(straat + ' ' + huisnummers + "\n" + postcode  + " Amsterdam")

info_adresses = info_city.groupby(["pand_id", "postcode"]).progress_apply(lambda x:get_info(x)).reset_index()

100%|████████████████████████████████████████████████████████████████████████████| 4211/4211 [00:01<00:00, 2250.02it/s]


In [20]:
def combine_info(rows):
    if len(rows)>1:
        adress = str()
        for row in rows[0]:
            street = row.split("\n")
            adress = adress + street[0] + " " + street[1].split(" ")[0] + "\n"
        adress = adress + "All in Amsterdam"
        return(adress)
    else:
        return(rows[0].values[0])    

info_adresses = info_adresses.groupby("pand_id").apply(lambda x:combine_info(x)).reset_index().rename(columns={0:"full_adress"})

In [21]:
info_building = info_city[["pand_id", "wgs"]].drop_duplicates()
info_building["wgs"] = info_building.wgs.apply(lambda x:literal_eval(x))
info_function = info_city.groupby("pand_id").gebruiksdoelVerblijfsobject.apply(lambda x: x.tolist()).reset_index()

In [22]:
city_info = pd.merge(info_adresses, info_building, on="pand_id", how="right").merge(info_function, on="pand_id", how="left")

In [23]:
city_info.head()

,pand_id,full_adress,wgs,gebruiksdoelVerblijfsobject
0,363100012061216.0,Kraaipanstraat 29\n1091PG Amsterdam,"[[52.35238128594593, 4.920002139317494], [52.3...",[woonfunctie]
1,363100012061274.0,Pretoriusstraat 85\n1092GD Amsterdam,"[[52.35448101198129, 4.921149991481547], [52.3...","[woonfunctie, woonfunctie, woonfunctie, woonfu..."
2,363100012061397.0,Oosterpark 16\n1092AG Amsterdam,"[[52.35813891596806, 4.918202870826519], [52.3...",[woonfunctie]
3,363100012061441.0,Hertzogstraat 7\n1092VS Amsterdam,"[[52.356361649110845, 4.9243825529648015], [52...","[woonfunctie, woonfunctie, woonfunctie, woonfu..."
4,363100012061506.0,Vrolikstraat 204\n1092TT Amsterdam,"[[52.357379461769796, 4.922498603589044], [52....","[woonfunctie, woonfunctie, woonfunctie, woonfu..."


## Calculate offset_small and offset_big for buildings

In [24]:
def get_offset(coordinates, radius):
    # transform radius to right size for clipper
    radius = radius/100000

    # do clipper calculations to get offset
    clipper_offset     = pyclipper.PyclipperOffset()
    coordinates_scaled = pyclipper.scale_to_clipper(coordinates)

    clipper_offset.AddPath(coordinates_scaled, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)

    new_coordinates    = clipper_offset.Execute(pyclipper.scale_to_clipper(radius))
    scaled_coordinates = pyclipper.scale_from_clipper(new_coordinates)
    
    return scaled_coordinates[0]

In [26]:
# small offset
city_info["offset_small"] = city_info.wgs.progress_apply(lambda x:get_offset(x, 10))
# big offset
city_info["offset_big"] = city_info.wgs.progress_apply(lambda x:get_offset(x, 25))
# offset for finding neighbors
city_info["offset_neighbors"] = city_info.wgs.progress_apply(lambda x:get_offset(x,1))

100%|█████████████████████████████████████████████████████████████████████████████| 3772/3772 [00:09<00:00, 399.47it/s]


## Find overlapping buildings

In [27]:
def get_overlap_polygons(offset, frame):
    poly = Polygon(offset)
    frame["intersect"] = frame.wgs.apply(lambda x:Polygon(x).intersects(poly))
    
    ids = [float(i) for i in frame[frame.intersect].pand_id.values]
    return ids

Next step can take about 15 minutes, progress is shown.

In [29]:
buildings = city_info[["pand_id", "wgs"]].copy()
city_info["neighbors"] = city_info.offset_neighbors.progress_apply(lambda x:get_overlap_polygons(x, buildings))
city_info["linked_small"] = city_info.offset_small.progress_apply(lambda x:get_overlap_polygons(x, buildings))
city_info["linked_big"] = city_info.offset_big.progress_apply(lambda x:get_overlap_polygons(x, buildings))

100%|██████████████████████████████████████████████████████████████████████████████| 3772/3772 [06:06<00:00, 10.29it/s]


In [45]:
# # remove id from own building
city_info.apply(lambda x:x.neighbors.remove(float(x.pand_id)), axis=1)
city_info.apply(lambda x:x.linked_small.remove(float(x.pand_id)), axis=1)
city_info.apply(lambda x:x.linked_big.remove(float(x.pand_id)), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
3767    None
3768    None
3769    None
3770    None
3771    None
Length: 3772, dtype: object

## Find overlapping tram and metro lines

In [46]:
def convert(test):
    test = test.replace('"', '')
    test2 = test.split(',')

    test3 = [x.split(' ') for x in test2]

    final = []
    for coord in test3:
        if len(coord)>2:
            coord = [float(c) for c in coord if c != ""]
        else:
            coord = [float(c) for c in coord]
        final.append(coord)
    return final

In [78]:
ov = pd.read_csv("tram en metro lijnen plus stations.csv")
ov["lijn_coordinaten"] = ov.lijn_coordinaten.apply(lambda x:convert(x))

In [48]:
def get_overlap_polygon_line(offset, frame):
    poly = Polygon(offset)
    frame["intersect"] = frame.lijn_coordinaten.apply(lambda x:LineString(x).intersects(poly))
    
    ids = [float(i) for i in frame[frame.intersect].number.values]
    return ids

In [50]:
city_info["ov_small"] = city_info.offset_small.progress_apply(lambda x:get_overlap_polygon_line(x, ov))
city_info["ov_big"] = city_info.offset_big.progress_apply(lambda x:get_overlap_polygon_line(x, ov))

100%|█████████████████████████████████████████████████████████████████████████████| 3772/3772 [00:35<00:00, 107.21it/s]


In [9]:
city_info.head()

,pand_id,full_adress,wgs,gebruiksdoelVerblijfsobject,offset_small,offset_big,offset_neighbors,neighbors,linked_small,linked_big,ov_small,ov_big,roads_small,roads_big
0,3.631000e+14,Eerste Boerhaavestraat 1\n1091RZ Amsterdam,"[[52.358244740379014, 4.906877228363365], [52....","['industry function', 'residential function', ...","[[52.35828490881249, 4.90665795141831], [52.35...","[[52.35828508576378, 4.90650795167312], [52.35...","[[52.35828463640064, 4.9067479516379535], [52....",[363100012160348.0],"[363100012160348.0, 363100012160950.0, 3631000...","[363100012159916.0, 363100012160348.0, 3631000...",[],[],[],[]
1,3.631000e+14,Eerste Boerhaavestraat 2\n1091SB Amsterdam,"[[52.357993489808614, 4.906806682187448], [52....","['residential function', 'other usage', 'resid...","[[52.357980388682336, 4.906641049776226], [52....","[[52.357980728615075, 4.906491050031036], [52....","[[52.35798009764403, 4.906731049530208], [52.3...","[363100012152835.0, 363100012165668.0]","[363100012152835.0, 363100012143801.0, 3631000...","[363100012236021.0, 363100012160348.0, 3631000...",[],[],[],[]
2,3.631000e+14,Eerste Boerhaavestraat 3\n1091RZ Amsterdam,"[[52.35826230344714, 4.906963883103816], [52.3...","['residential function', 'residential function...","[[52.35824508219957, 4.906777228694409], [52.3...","[[52.358245245181024, 4.906627228949219], [52....","[[52.358244818169624, 4.906867228914052], [52....","[363100012236021.0, 363100012160950.0]","[363100012236021.0, 363100012160950.0, 3631000...","[363100012236021.0, 363100012159916.0, 3631000...",[],[],[],[]
3,3.631000e+14,Eerste Boerhaavestraat 4\n1091SB Amsterdam,"[[52.358007051008784, 4.906872813644981], [52....","['industry function', 'residential function', ...","[[52.35799377691001, 4.906706682406366], [52.3...","[[52.3579938900657, 4.906556682661176], [52.35...","[[52.35799358272925, 4.906796682626009], [52.3...","[363100012159916.0, 363100012143801.0]","[363100012159916.0, 363100012143801.0, 3631000...","[363100012236021.0, 363100012159916.0, 3631000...",[],[],[],[]
4,3.631000e+14,Eerste Boerhaavestraat 5\n1091RZ Amsterdam,"[[52.35827993971986, 4.90705087475251], [52.35...","['residential function', 'residential function...","[[52.35826265066862, 4.906863883603364], [52.3...","[[52.358262822031975, 4.906713883858174], [52....","[[52.35826238198206, 4.906953883823007], [52.3...","[363100012160348.0, 363100012156395.0]","[363100012236021.0, 363100012160348.0, 3631000...","[363100012236021.0, 363100012160348.0, 3631000...",[],[],[],[]


## replace funtion names

In [51]:
def translate_functions(functions):
    
    mapper = {
        "woonfunctie":"residential function",
        "winkelfunctie":"shopping function",
        "industriefunctie":"industry function",
        "bijeenkomstfunctie":"meet function",
        "kantoorfunctie":"office function",
        "overige gebruiksfunctie":"other usage",
        "onderwijsfunctie":"educational function",
        "gezondheidszorgfunctie":"health care function",
        "sportfunctie":"sports function",
        "logiesfunctie":"accomodation function",
        "celfunctie":"cell function"
    }
    
    functions = [mapper[function] for function in functions]
    
    return functions

In [53]:
city_info["gebruiksdoelVerblijfsobject"] = city_info.gebruiksdoelVerblijfsobject.apply(lambda x:translate_functions(x))

## find overlapping roads

In [55]:
df_roads = pd.read_csv("PLUSHOOFDNETTEN.csv", delimiter=";")
df_roads = df_roads[['OBJECTNUMMER', "STT_NAAM", "AUTO", "WKT_LAT_LNG"]][df_roads.AUTO.notnull()]
df_roads['WKT_LAT_LNG'] = df_roads.WKT_LAT_LNG.apply(lambda x:x.replace("LINESTRING(", "").replace(")", ""))

In [56]:
df_emergency = pd.read_csv("emergency_routes.csv")
df_emergency['AUTO'] = "calamiteit"

In [59]:
# merge the two road csv's
all_roads = pd.concat([df_roads.drop(columns="OBJECTNUMMER"), df_emergency[['STT_NAAM', 'AUTO', 'WKT_LAT_LNG']]]).reset_index().drop(columns='index')
# renaming to two entities
all_roads['AUTO'] = all_roads.AUTO.replace({'PLUS':'plus', 'HOOFD':'hoofd', 'CORRIDOR':'plus', 'PLUS_BGG':'plus', 'CORRIDOR_B':'plus'})
# create number for each road as identifier
all_roads['number'] = all_roads.index
# make from strings a list
all_roads["WKT_LAT_LNG"] = all_roads.WKT_LAT_LNG.apply(convert)

In [60]:
def get_overlap_polygon_line(offset, frame):
    poly = Polygon(offset)
    frame["intersect"] = frame.WKT_LAT_LNG.apply(lambda x:LineString(x).intersects(poly))
    
    ids = [float(i) for i in frame[frame.intersect].number.values]
    return ids

In [61]:
city_info['roads_small'] = city_info.offset_small.progress_apply(lambda x:get_overlap_polygon_line(x, all_roads))
city_info['roads_big'] = city_info.offset_big.progress_apply(lambda x:get_overlap_polygon_line(x, all_roads))

100%|██████████████████████████████████████████████████████████████████████████████| 3772/3772 [03:00<00:00, 20.84it/s]


## calculate score fore three focusses

In [85]:
def ov_score(ov_numbers, mapper, frame=ov):
    
    # get ov ids
#     ov_numbers = literal_eval(ov_numbers)
    ov_ids = [int(number) for number in ov_numbers]
    
    # get their modality
    types = list(frame[frame.number.isin(ov_ids)].modaliteit.values)
    
    # map to score
    ov_score = sum([mapper[t] for t in types])
    return ov_score

def road_score(road_numbers, mapper, frame=all_roads):
    
    # get road ids
#     road_numbers = literal_eval(road_numbers)
    road_ids = [int(number) for number in road_numbers]
    
    # get their modality
    types = list(frame[frame.number.isin(road_ids)].AUTO.values)
    
    # map to score
    road_score = sum([mapper[t] for t in types])
    return road_score

def function_score(functions, buildings, mapper, frame=city_info):
    
    # get functions of buildings in radius
#     ids = literal_eval(buildings)
    radius_functions = list(frame[frame.pand_id.isin(buildings)].gebruiksdoelVerblijfsobject.explode().values)
    
    # add functions of own building
    all_functions = radius_functions + functions
    
    # map to score
    function_score = sum([mapper[function] for function in all_functions])
    return function_score

In [86]:
def get_score_small(row, ov_mapper, road_mapper, function_mapper):
    score_ov = ov_score(row.ov_small, ov_mapper)
    score_road = road_score(row.roads_small, road_mapper)
    score_building = function_score(row.gebruiksdoelVerblijfsobject, row.linked_small, function_mapper)
    
    total_score = score_ov + score_road + score_building
    return total_score
    
def get_score_big(row, ov_mapper, road_mapper, function_mapper):
    score_ov = ov_score(row.ov_big, ov_mapper)
    score_road = road_score(row.roads_big, road_mapper)
    score_building = function_score(row.gebruiksdoelVerblijfsobject, row.linked_big, function_mapper)
    
    total_score = score_ov + score_road + score_building
    return total_score

In [87]:
scaler = MinMaxScaler()

### general

In [88]:
general_ov_mapper = {
    'Tram' :3,
    'Metro':4
}

general_function_mapper = {
    'other usage' : 0, 
    'shopping function' : 1,
    'sports function' : 1, 
    'accomodation function' : 2, 
    'cell function' : 3,
    'industry function' : 2,
    'office function' : 1, 
    'residential function' : 0,
    'meet function' : 1, 
    'educational function' : 2, 
    'health care function' : 3
}

general_road_mapper = {
    'calamiteit':3,
    'hoofd':3,
    'plus':3
}

In [89]:
city_info['score_small_default'] = city_info.progress_apply(lambda x:get_score_small(x, general_ov_mapper, general_road_mapper, general_function_mapper), axis=1)
city_info['score_big_default'] = city_info.progress_apply(lambda x:get_score_big(x, general_ov_mapper, general_road_mapper, general_function_mapper), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████| 3772/3772 [00:07<00:00, 514.30it/s]


In [107]:
df_scaled = pd.DataFrame(scaler.fit_transform(city_info[['score_small_default', 'score_big_default']]), columns=['score_small_default', 'score_big_default'])
city_info = pd.concat([city_info, df_scaled], axis=1)

### residential focus

In [108]:
residential_function_mapper = { 
    'other usage' : 1, 
    'shopping function' : 1,
    'sports function' : 1, 
    'accomodation function' : 1, 
    'cell function' : 2,
    'industry function' : 1,
    'office function' : 1, 
    'residential function' : 6,
    'meet function' : 1, 
    'educational function' : 1, 
    'health care function' : 2
} 

residential_ov_mapper = { 
    'Tram' : 3, 
    'Metro' : 4 
} 

residential_road_mapper = { 
    'hoofd' : 3, 
    'plus' : 3, 
    'calamiteit' : 3 
} 

In [109]:
city_info['score_small_residential'] = city_info.progress_apply(lambda x:get_score_small(x, residential_ov_mapper, residential_road_mapper, residential_function_mapper), axis=1)
city_info['score_big_residential'] = city_info.progress_apply(lambda x:get_score_big(x, residential_ov_mapper, residential_road_mapper, residential_function_mapper), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████| 3772/3772 [00:07<00:00, 518.97it/s]


In [110]:
df_scaled = pd.DataFrame(scaler.fit_transform(city_info[['score_small_residential', 'score_big_residential']]), columns=['score_small_residential', 'score_big_residential'])
city_info = pd.concat([city_info, df_scaled], axis=1)

### roads focus

In [111]:
road_function_mapper = { 
    'other usage' : 0,  
    'shopping function' : 1, 
    'sports function' : 1,  
    'accomodation function' : 2,  
    'cell function' : 3, 
    'industry function' : 2, 
    'office function' : 1,  
    'residential function' : 0, 
    'meet function' : 1,  
    'educational function' : 2,  
    'health care function' : 3 
} 

road_ov_mapper = { 
    'Tram' : 5, 
    'Metro' : 6 
} 

road_road_mapper = { 
    'hoofd' : 5, 
    'plus' : 5, 
    'calamiteit' : 5 
} 

In [112]:
city_info['score_small_road'] = city_info.progress_apply(lambda x:get_score_small(x, road_ov_mapper, road_road_mapper, road_function_mapper), axis=1)
city_info['score_big_road'] = city_info.progress_apply(lambda x:get_score_big(x, road_ov_mapper, road_road_mapper, road_function_mapper), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████| 3772/3772 [00:07<00:00, 515.82it/s]


In [113]:
df_scaled = pd.DataFrame(scaler.fit_transform(city_info[['score_small_road', 'score_big_road']]), columns=['score_small_road', 'score_big_road'])
city_info = pd.concat([city_info, df_scaled], axis=1)

# Final dataframe

In [115]:
city_info.head()

,pand_id,full_adress,wgs,gebruiksdoelVerblijfsobject,offset_small,offset_big,offset_neighbors,neighbors,linked_small,linked_big,...,score_small_default,score_big_default,score_small_residential,score_big_residential,score_small_residential,score_big_residential,score_small_road,score_big_road,score_small_road,score_big_road
0,363100012061216.0,Kraaipanstraat 29\n1091PG Amsterdam,"[[52.35238128594593, 4.920002139317494], [52.3...",[residential function],"[[52.35233497899026, 4.919793281238526], [52.3...","[[52.35233529750258, 4.919643281493336], [52.3...","[[52.352355615701526, 4.9198925332166255], [52...","[363100012103446.0, 363100012154076.0]","[363100012086048.0, 363100012103446.0, 3631000...","[363100012061600.0, 363100012062562.0, 3631000...",...,0.0,0.000000,6,6,0.005102,0.005071,0,0,0.0,0.000000
1,363100012061274.0,Pretoriusstraat 85\n1092GD Amsterdam,"[[52.35448101198129, 4.921149991481547], [52.3...","[residential function, residential function, r...","[[52.35445520747453, 4.920959553681314], [52.3...","[[52.35445553762838, 4.920809554401785], [52.3...","[[52.354454963002354, 4.921049553900957], [52....","[363100012063216.0, 363100012087223.0]","[363100012063216.0, 363100012068336.0, 3631000...","[363100012062237.0, 363100012063216.0, 3631000...",...,0.0,0.000000,24,24,0.023469,0.023327,0,0,0.0,0.000000
2,363100012061397.0,Oosterpark 16\n1092AG Amsterdam,"[[52.35813891596806, 4.918202870826519], [52.3...",[residential function],"[[52.358118620701134, 4.918022162280977], [52....","[[52.358118715230376, 4.917872162070125], [52....","[[52.35811832174659, 4.91811216250062], [52.35...","[363100012134220.0, 363100012157417.0]","[363100012134220.0, 363100012143269.0, 3631000...","[363100012061601.0, 363100012070318.0, 3631000...",...,0.0,0.122449,6,12,0.005102,0.011156,0,10,0.0,0.123457
3,363100012061441.0,Hertzogstraat 7\n1092VS Amsterdam,"[[52.356361649110845, 4.9243825529648015], [52...","[residential function, residential function, r...","[[52.35637346049771, 4.924144277349114], [52.3...","[[52.35637373710051, 4.923994277603924], [52.3...","[[52.35637311870232, 4.924234277103096], [52.3...","[363100012136218.0, 363100012233692.0]","[363100012136218.0, 363100012233692.0, 3631000...","[363100012082397.0, 363100012085865.0, 3631000...",...,0.0,0.000000,30,30,0.029592,0.029412,0,0,0.0,0.000000
4,363100012061506.0,Vrolikstraat 204\n1092TT Amsterdam,"[[52.357379461769796, 4.922498603589044], [52....","[residential function, residential function, r...","[[52.35735902283341, 4.9223246751353145], [52....","[[52.35735957277939, 4.922174676321447], [52.3...","[[52.35735892597586, 4.922414675820619], [52.3...","[363100012092987.0, 363100012158883.0]","[363100012062310.0, 363100012063951.0, 3631000...","[363100012062009.0, 363100012062310.0, 3631000...",...,0.0,0.000000,30,30,0.029592,0.029412,0,0,0.0,0.000000


In [117]:
# remove offsets since they only require storage and are fastly calculated in the system
city_info = city_info.drop(columns=['offset_small', 'offset_big', 'offset_neighbors'])

In [118]:
# write df to csv for use in the system
city_info.to_csv("city_buildings.csv", index=False)